In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark OpenStreetMap project") \
    .config("spark.some.config.option", "osm-hungary") \
    .getOrCreate()

In [3]:
hungaryNodes = spark.read.parquet("parquets/hungary-latest.osm.pbf.node.parquet")
hungaryWays = spark.read.parquet("parquets/hungary-latest.osm.pbf.way.parquet")
hungaryRelations = spark.read.parquet("parquets/hungary-latest.osm.pbf.relation.parquet")

In [4]:
hungaryNodes.createOrReplaceTempView("hungaryNodes")
hungaryWays.createOrReplaceTempView("hungaryWays")
hungaryRelations.createOrReplaceTempView("hungaryRelations")

In [5]:
from pyspark.sql.functions import col, explode, array_contains, collect_list, struct

In [6]:
from pyspark.sql.types import ArrayType, StringType

In [7]:
search_key = 'highway'

In [8]:
wayNode_df = hungaryWays.filter(array_contains(col('tags').getField('key')\
                                               .cast(ArrayType(StringType())), search_key))\
.select(col('id').alias('wayId'), col('user_sid'), 
        col('tags').getField('key').cast(ArrayType(StringType())).alias('keys'),\
        col('tags').getField('value').cast(ArrayType(StringType())).alias('values'),\
        explode("nodes").alias('indexedNode') )

In [9]:
wayNode_df = wayNode_df.withColumn('user_sid', wayNode_df['user_sid'].cast(StringType()))

In [10]:
wayNode_df.show(10)

+-------+--------+---------------+--------------------+---------------+
|  wayId|user_sid|           keys|              values|    indexedNode|
+-------+--------+---------------+--------------------+---------------+
|3175810| SzPaula|[highway, name]|[residential, Hon...|  [0, 15231786]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[1, 1310148452]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[2, 1310021025]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[3, 1310020985]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[4, 1310020988]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[5, 1310021030]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[6, 1310021038]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[7, 1310021044]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[8, 1309949162]|
|3175810| SzPaula|[highway, name]|[residential, Hon...|[9, 1309949099]|
+-------+--------+---------------+--------------------+---------

In [11]:
import spark_df_profiling

In [14]:
wayNode_df = wayNode_df.withColumn('keys', wayNode_df['keys'].cast(StringType()))

In [15]:
wayNode_df = wayNode_df.withColumn('values', wayNode_df['values'].cast(StringType()))

In [16]:
wayNode_df = wayNode_df.withColumn('indexedNode', wayNode_df['indexedNode'].cast(StringType()))

In [17]:
report = spark_df_profiling.ProfileReport(wayNode_df)

C:\ProgramData\Anaconda34\envs\gpu_env\lib\site-packages\spark_df_profiling\base.py:164: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.
  plot.set_axis_bgcolor("w")


In [18]:
report

Number of variables,5
Number of observations,4809092
Total Missing (%),0.0%
Total size in memory,0.0 B
Average record size in memory,0.0 B
Numeric,1
Categorical,4
Date,0
Text (Unique),0
Rejected,0
Distinct count,509675


In [ ]:
nodCoor_df = hungaryNodes\
    .select(col('id').alias('nodeId'), col('id'), col('latitude'), col('longitude'))

In [ ]:
nodCoor_df.show(10)

In [ ]:
a = wayNode_df.indexedNode.getField('nodeId').cast(StringType())

In [ ]:
b = nodCoor_df.nodeId.cast(StringType())

In [ ]:
way_geometry_df = wayNode_df.join(nodCoor_df, a==b)

In [ ]:
way_geometry_df.show(10)

In [ ]:
way_geometry_df = way_geometry_df.groupBy('wayId', 'values')\
                                 .agg(collect_list(struct(col('indexedNode').getField('index'),\
                                                          col('latitude'),\
                                                          col('longitude'))).alias('geometry'))


In [ ]:
way_geometry_df.show(10)

In [ ]:
way_geometry_df = way_geometry_df.limit(1000).toPandas()

In [ ]:
way_geometry_df.head()

In [ ]:
way_geometry_df.iloc[0]['geometry']

In [ ]:
import plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go
import pandas as pd

In [ ]:
data = dict (
    type = 'choropleth',
    locations = ['Hungary'],
    locationmode='country names',
    colorscale = 'Viridis',
    z=[50])

In [ ]:
lyt = dict(geo=dict(scope='europe'))
fig = go.Figure(data=[data], layout = lyt)

plotly.offline.iplot(fig)

In [ ]:
temp = way_geometry_df.iloc[0]['geometry']

In [ ]:
temp[0]

In [ ]:
len(way_geometry_df)

In [ ]:

flight_paths = []
for j in range(len(way_geometry_df)):
    for i in range(len(way_geometry_df.iloc[j]['geometry'])):
        flight_paths.append(
            go.Scattergeo(
                locationmode = 'country names',
                lon = [way_geometry_df.iloc[j]['geometry'][i]['longitude'], way_geometry_df.iloc[j]['geometry'][int(i+1) % len(way_geometry_df.iloc[j]['geometry'])]['longitude']],
                lat = [way_geometry_df.iloc[j]['geometry'][i]['latitude'], way_geometry_df.iloc[j]['geometry'][int(i+1) % len(way_geometry_df.iloc[j]['geometry'])]['latitude']],
                mode = 'lines',
                line = go.scattergeo.Line(
                    width = 10,
                    color = 'red',
                ),
            opacity =1,
            )
        )

layout = go.Layout(
    title = go.layout.Title(
        text = 'Highways, Hungary'
    ),
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'europe',
        projection = go.layout.geo.Projection(type = 'azimuthal equal area'),
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig = go.Figure(data = flight_paths, layout = layout)
plotly.offline.iplot(fig, filename = 'highways')